<a href="https://colab.research.google.com/github/mehrdadrashidian/CG-Ecosystem/blob/master/Cartoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask flask-ngrok tensorflow opencv-python numpy

In [3]:
# 👐 نصب و پیکربندی Kaggle API (اگر برای اولین بار اجرا می‌کنید)
!pip install kaggle -q

# 👐 ساخت پوشه برای API Key و تنظیمات Kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"mehrdadrashidian","key":"ce0eed3e55694d36a5012de688bfe7d3"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# 👐 ساخت پوشه‌های موردنیاز قبل از استخراج فایل‌ها
!mkdir -p /content/dataset/real_faces/
!mkdir -p /content/dataset/cartoon_faces/

# 👐 دانلود دیتاست‌های صورت واقعی و کارتونی از Kaggle
!kaggle datasets download -d ashwingupta3012/human-faces -p /content/
!unzip /content/human-faces.zip -d /content/dataset/real_faces/

!kaggle datasets download -d splcher/animefacedataset -p /content/
!unzip /content/animefacedataset.zip -d /content/dataset/cartoon_faces/

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/cartoon_faces/images/62764_2019.jpg  
  inflating: /content/dataset/cartoon_faces/images/62765_2019.jpg  
  inflating: /content/dataset/cartoon_faces/images/62766_2019.jpg  
  inflating: /content/dataset/cartoon_faces/images/62767_2019.jpg  
  inflating: /content/dataset/cartoon_faces/images/62768_2019.jpg  
  inflating: /content/dataset/cartoon_faces/images/62769_2019.jpg  
  inflating: /content/dataset/cartoon_faces/images/6276_2003.jpg  
  inflating: /content/dataset/cartoon_faces/images/62770_2019.jpg  
  inflating: /content/dataset/cartoon_faces/images/62771_2019.jpg  
  inflating: /content/dataset/cartoon_faces/images/62772_2019.jpg  
  inflating: /content/dataset/cartoon_faces/images/62773_2019.jpg  
  inflating: /content/dataset/cartoon_faces/images/62774_2019.jpg  
  inflating: /content/dataset/cartoon_faces/images/62775_2019.jpg  
  inflating: /content/dataset/cartoon_faces/images/62776_2019.jpg 

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 👐 مسیر جدید دیتاست‌های دانلود شده
REAL_FACES_PATH = "/content/dataset/real_faces/Humans/"
CARTOON_FACES_PATH = "/content/dataset/cartoon_faces/images/"

# 👐 تابع بارگذاری دسته‌ای تصاویر
def load_images_in_batches(real_folder_path, cartoon_folder_path, batch_size=16, img_size=(256, 256)):
    real_image_paths = [os.path.join(real_folder_path, img_name) for img_name in os.listdir(real_folder_path)]
    cartoon_image_paths = [os.path.join(cartoon_folder_path, img_name) for img_name in os.listdir(cartoon_folder_path)]

    datagen = ImageDataGenerator(rescale=1./255)  # نرمال‌سازی تصاویر
    real_datagen = datagen.flow_from_directory(
        real_folder_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=True
    )
    cartoon_datagen = datagen.flow_from_directory(
        cartoon_folder_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=True
    )

    while True:  # هر بار یک دسته داده باز می‌گرداند
        real_images = next(real_datagen)
        cartoon_images = next(cartoon_datagen)
        yield real_images, cartoon_images

# 👐 بارگذاری دیتاست‌ها به صورت دسته‌ای
train_generator = load_images_in_batches(REAL_FACES_PATH, CARTOON_FACES_PATH)

# 👐 تعداد مراحل در هر اپوک
steps_per_epoch = len(os.listdir(REAL_FACES_PATH)) // 16  # تعداد دسته‌ها در هر اپوک

# 👐 تعریف مدل تبدیل عکس به کارتونی
def build_cartoon_model():
    model = Sequential([
        Conv2D(64, (3, 3), padding='same', input_shape=(256, 256, 3)),  # ابعاد ورودی تغییر کرده
        BatchNormalization(),
        Activation('relu'),
        Conv2D(3, (3, 3), padding='same', activation='sigmoid')  # حذف UpSampling2D برای مطابقت ابعاد
    ])
    model.compile(optimizer='adam', loss='MeanSquaredError')
    return model

# 👐 ساخت مدل و آموزش آن با داده‌های دسته‌ای
cartoon_model = build_cartoon_model()

# 🏋️‍♀️ آموزش مدل با استفاده از داده‌های دسته‌ای
cartoon_model.fit(
    train_generator,
    epochs=12,
    steps_per_epoch=steps_per_epoch,
    validation_data=train_generator,  # اینجا برای سادگی از همان generator استفاده می‌کنیم
    validation_steps=steps_per_epoch
)

# 👐 ذخیره مدل
MODEL_PATH = "/content/cartoon_model.h5"
cartoon_model.save(MODEL_PATH)
print(f"✅ مدل ذخیره شد: {MODEL_PATH}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Epoch 1/12
451/451 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/12
451/451 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/12
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/12
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/12
451/451 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/12
451/451 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/12
451/451 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/12
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/12
451/451 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 10/12
451/451 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0000e+00 - 

✅ مدل ذخیره شد: /content/cartoon_model.h5


In [6]:
from google.colab import files
files.download('/content/cartoon_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
!pip install Flask tensorflow opencv-python Pillow

In [ ]:
from flask import Flask, render_template, request, send_file
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import MeanSquaredError
import numpy as np
import io
from PIL import Image

# بارگذاری مدل با custom_objects
model = load_model('/content/cartoon_model.h5', custom_objects={'mse': MeanSquaredError()})

# ایجاد اپلیکیشن فلکس
app = Flask(__name__)

# صفحه اصلی که از کاربر عکس می‌گیرد
@app.route('/')
def home():
    return '''
    <!DOCTYPE html>
    <html lang="fa">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>تبدیل عکس به انیمه</title>
    </head>
    <body>
        <h1>آپلود عکس خود را انجام دهید</h1>
        <form action="/predict" method="POST" enctype="multipart/form-data">
            <input type="file" name="image" accept="image/*" required>
            <button type="submit">ارسال تصویر</button>
        </form>
    </body>
    </html>
    '''

# دریافت تصویر از کاربر و پردازش آن
@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return "No file part"

    file = request.files['image']
    if file.filename == '':
        return "No selected file"

    # تبدیل فایل به آرایه numpy
    img = Image.open(file.stream)
    img = img.resize((128, 128))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    # پیش‌بینی تصویر انیمه
    cartoon_img = model.predict(img)
    cartoon_img = np.squeeze(cartoon_img, axis=0)
    cartoon_img = (cartoon_img * 255).astype(np.uint8)

    # تبدیل به فرمت قابل نمایش
    img_pil = Image.fromarray(cartoon_img)
    img_io = io.BytesIO()
    img_pil.save(img_io, 'PNG')
    img_io.seek(0)

    # ارسال تصویر انیمه به کاربر
    return send_file(img_io, mimetype='image/png')

# اجرای اپلیکیشن Flask
if __name__ == "__main__":
    app.run(debug=True)


In [7]:
!pip install pyngrok


In [12]:
from flask import Flask, render_template, request, send_file
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import MeanSquaredError
import numpy as np
import io
from PIL import Image
from pyngrok import ngrok

# توکن Ngrok خود را وارد کنید
NGROK_TOKEN = '2rBB8oZuHhqKW1cfZb4yV4ZKYtb_59fGYc2gwf6PQuPrDuuFE'

# اتصال به Ngrok
ngrok.set_auth_token(NGROK_TOKEN)
public_url = ngrok.connect(5000)
print(f"{public_url}")

# بارگذاری مدل با custom_objects
model = load_model('/content/cartoon_model.h5', custom_objects={'mse': MeanSquaredError()})

# ایجاد اپلیکیشن فلکس
app = Flask(__name__)

# صفحه اصلی که از کاربر عکس می‌گیرد
@app.route('/')
def home():
    return '''
    <!DOCTYPE html>
    <html lang="fa">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>تبدیل عکس به انیمه</title>
    </head>
    <body>
        <h1>آپلود عکس خود را انجام دهید</h1>
        <form action="/predict" method="POST" enctype="multipart/form-data">
            <input type="file" name="image" accept="image/*" required>
            <button type="submit">ارسال تصویر</button>
        </form>
    </body>
    </html>
    '''

# دریافت تصویر از کاربر و پردازش آن
@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return "No file part"

    file = request.files['image']
    if file.filename == '':
        return "No selected file"

    # تبدیل فایل به آرایه numpy
    img = Image.open(file.stream)
    img = img.resize((128, 128))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    # پیش‌بینی تصویر انیمه
    cartoon_img = model.predict(img)
    cartoon_img = np.squeeze(cartoon_img, axis=0)
    cartoon_img = (cartoon_img * 255).astype(np.uint8)

    # تبدیل به فرمت قابل نمایش
    img_pil = Image.fromarray(cartoon_img)
    img_io = io.BytesIO()
    img_pil.save(img_io, 'PNG')
    img_io.seek(0)

    # ارسال تصویر انیمه به کاربر
    return send_file(img_io, mimetype='image/png')

# اجرای اپلیکیشن Flask
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [1]:
from google.colab import files
uploaded = files.upload()


Saving cartoon_model.h5 to cartoon_model.h5


In [2]:
!ls /content


cartoon_model.h5  sample_data


In [4]:
!pip install pyngrok


In [6]:
from flask import Flask, render_template, request, send_file
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import MeanSquaredError
import numpy as np
import io
from PIL import Image
import os
from pyngrok import ngrok

# توکن Ngrok خود را وارد کنید
NGROK_TOKEN = '2rBB8oZuHhqKW1cfZb4yV4ZKYtb_59fGYc2gwf6PQuPrDuuFE'

# اتصال به Ngrok
ngrok.set_auth_token(NGROK_TOKEN)
public_url = ngrok.connect(5000)
print(f"{public_url}")

# بارگذاری مدل با custom_objects
model = load_model('/content/cartoon_model.h5', custom_objects={'mse': MeanSquaredError()})
# ایجاد اپلیکیشن فلکس
app = Flask(__name__)

# صفحه اصلی که از کاربر عکس می‌گیرد
@app.route('/')
def home():
    return '''
    <!DOCTYPE html>
    <html lang="fa">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>تبدیل عکس به انیمه</title>
    </head>
    <body>
        <h1>آپلود عکس خود را انجام دهید</h1>
        <form action="/predict" method="POST" enctype="multipart/form-data">
            <input type="file" name="image" accept="image/*" required>
            <button type="submit">ارسال تصویر</button>
        </form>
    </body>
    </html>
    '''

# دریافت تصویر از کاربر و پردازش آن
@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return "No file part"

    file = request.files['image']
    if file.filename == '':
        return "No selected file"

    # تبدیل فایل به آرایه numpy
    img = Image.open(file.stream)
    img = img.resize((128, 128))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    # پیش‌بینی تصویر انیمه
    cartoon_img = model.predict(img)

    # بررسی ابعاد تصویر پیش‌بینی شده
    print(f"Shape of predicted image: {cartoon_img.shape}")

    # اطمینان از اینکه خروجی به صورت درست تغییر داده شده باشد
    cartoon_img = np.squeeze(cartoon_img, axis=0)
    cartoon_img = (cartoon_img * 255).astype(np.uint8)

    # تبدیل به فرمت قابل نمایش
    img_pil = Image.fromarray(cartoon_img)

    # بررسی نمایش تصویر قبل از ارسال
    img_pil.show()

    # تبدیل تصویر به بایت‌ها برای ارسال به کاربر
    img_io = io.BytesIO()
    img_pil.save(img_io, 'PNG')
    img_io.seek(0)

    # ارسال تصویر انیمه به کاربر
    return send_file(img_io, mimetype='image/png')

# اجرای اپلیکیشن Flask
if __name__ == "__main__":
    app.run(debug=True)


NgrokTunnel: "https://5d5d-34-73-0-124.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
